In [1]:
import neo4j_utils

import os
import pandas as pd

COMPILED_GRAPH_DIRECTORY = "tmp/compiled_graphs"

/Users/minerva/Desktop/Neo-Project/Neo-KnowledgePass/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
all_course_job_paths = neo4j_utils.find_all_possible_course_job_path()
len(all_course_job_paths)

3

In [3]:
rows = []

for k in all_course_job_paths.keys():
    for v in all_course_job_paths[k]:
        relations = list(v.get('p'))

        path = []

        path.append(relations[0].start_node.get('name'))

        for r in relations:
            start_node = r.start_node.get('name')
            end_node = r.end_node.get('name')

            if path[-1] != end_node:
                path.append(f'{r.type}>')
                path.append(end_node)
            else:
                path.append(f'{r.type}<')
                path.append(start_node)

        rows.append({
            'type': k,
            'job': path[0],
            'job_require': path[2],
            'course_acquire': path[-3],
            'course': path[-1],
            'path': path
        })

df = pd.DataFrame(rows)
df

,type,job,job_require,course_acquire,course,path
0,single_hop_synonym,Backend (Travel) #1,Computer Science,Computer Science,SF220,"[Backend (Travel) #1, require>, Computer Scien..."
1,single_hop_synonym,Backend (Travel) #1,Deployment,Deployment,SF222,"[Backend (Travel) #1, require>, Deployment, in..."
2,single_hop_synonym,Backend (Travel) #1,Deployment,Deployment,SF220,"[Backend (Travel) #1, require>, Deployment, in..."
3,single_hop_synonym,Backend (Travel) #1,Git,Git,SF222,"[Backend (Travel) #1, require>, Git, include<,..."
4,single_hop_synonym,Backend (Travel) #1,Git,Git,CN310,"[Backend (Travel) #1, require>, Git, include<,..."
...,...,...,...,...,...,...
6515,subclass_of,Fullstack (MedTech) #1,Nest.JS,Software Engineering,SF230,"[Fullstack (MedTech) #1, require>, Nest.JS, su..."
6516,subclass_of,Fullstack (MedTech) #1,Nest.JS,Software Engineering,SF220,"[Fullstack (MedTech) #1, require>, Nest.JS, su..."
6517,subclass_of,Fullstack (MedTech) #1,Full Stack Engineer,Software Engineering,CN310,"[Fullstack (MedTech) #1, require>, Full Stack ..."
6518,subclass_of,Fullstack (MedTech) #1,Full Stack Engineer,Software Engineering,SF230,"[Fullstack (MedTech) #1, require>, Full Stack ..."


In [4]:
df[df.type=='subclass_of'].iloc[46].path

['Backend (Travel) #1',
 'require>',
 'Git',
 'subclass_of>',
 'Version Control Systems',
 'subclass_of>',
 'Software Development',
 'include<',
 'SF222']

In [2]:
'''
SQL is synonym of Sdskfjskdlfjd.
SQL is subclass of query lang
SQL use knowledge of database
'''

rows = []

node2text = neo4j_utils.node_to_text()

for k in node2text.keys():
    for v in node2text[k]:
        relations = list(v.get('p'))

        path = []

        path.append(relations[0].start_node.get('name'))

        for r in relations:
            start_node = r.start_node.get('name')
            end_node = r.end_node.get('name')

            path.append(k)
            path.append(end_node)

        rows.append({
            'type': k,
            'start_node': path[0],
            'end_node': path[-1],
            'text': ' '.join(path),
            'path': path
        })

sdf = pd.DataFrame(rows)
sdf

,type,start_node,end_node,text,path
0,is synonym of,Inventory Management,Inventory and Pricing,Inventory Management is synonym of Inventory a...,"[Inventory Management, is synonym of, Inventor..."
1,is synonym of,Pricing Strategies,Inventory and Pricing,Pricing Strategies is synonym of Inventory and...,"[Pricing Strategies, is synonym of, Inventory ..."
2,is synonym of,Product Data,Product Information,Product Data is synonym of Product Information,"[Product Data, is synonym of, Product Informat..."
3,is synonym of,Product Catalog,Product Information,Product Catalog is synonym of Product Information,"[Product Catalog, is synonym of, Product Infor..."
4,is synonym of,Customer Information,Customer Data,Customer Information is synonym of Customer Data,"[Customer Information, is synonym of, Customer..."
...,...,...,...,...,...
9219,use knowledge of,Network Environment,Ethernet,Network Environment use knowledge of Ethernet,"[Network Environment, use knowledge of, Ethernet]"
9220,use knowledge of,Network Environment,Node,Network Environment use knowledge of Node,"[Network Environment, use knowledge of, Node]"
9221,use knowledge of,Streaming Server,Stream,Streaming Server use knowledge of Stream,"[Streaming Server, use knowledge of, Stream]"
9222,use knowledge of,Streaming Server,Encoder,Streaming Server use knowledge of Encoder,"[Streaming Server, use knowledge of, Encoder]"


In [3]:
grp = sdf.groupby('start_node')
len(grp)

3783

In [4]:
rows = []

for k, g in grp:
    lv1r = []

    for i, r in g.iterrows():
        lv1r.append(r['text'])

    rows.append({
        'node': k,
        'lv1_relation_text': lv1r
    })

ssdf = pd.DataFrame(rows)
ssdf

,node,lv1_relation_text
0,.NET Framework,[.NET Framework is synonym of .Net]
1,.deb,[.deb is synonym of Debian binary package]
2,2D Structures,"[2D Structures is subclass of Data Structures,..."
3,32kadpcm (32 kbps coding),[32kadpcm (32 kbps coding) is subclass of MIME...
4,3rd Party Library,[3rd Party Library is subclass of Library]
...,...,...
3778,while loops,[while loops is subclass of loops]
3779,whois,[whois is subclass of Install extra-package fo...
3780,working directory,[working directory is subclass of Version Cont...
3781,write (str),[write (str) is subclass of File Input/output ...


In [7]:
ssdf.to_csv('node2embedding.csv', index=False)

In [6]:
df.to_csv(os.path.join(COMPILED_GRAPH_DIRECTORY, 'course_job.csv'), index=False)

In [ ]:
import neo4j_utils

import os
import pandas as pd

COMPILED_GRAPH_DIRECTORY = "tmp/compiled_graphs"

# Precompute all course to job paths
all_course_job_paths = neo4j_utils.find_all_possible_course_job_path()

rows = []
for path in all_course_job_paths:
    row = {
        "course": path["course"]["name"],
        "course_skill": path["course_skill"]["name"],
        "rep_skill": path["rep_skill"]["name"],
        "job_skill": path["job_skill"]["name"],
        "job": path["job"]["name"]
    }
    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv(os.path.join(COMPILED_GRAPH_DIRECTORY, "course_job.csv"), index=False)

    # Precompute all skill to job paths
all_skill_job_paths = neo4j_utils.find_all_possible_skill_job_path()

rows = []
for path in all_skill_job_paths:
    row = {
        "skill": path["skill"]["name"],
        "rep_skill": path["rep_skill"]["name"],
        "job_skill": path["job_skill"]["name"],
        "job": path["job"]["name"]
    }
    rows.append(row)
df = pd.DataFrame(rows)
df.to_csv(os.path.join(COMPILED_GRAPH_DIRECTORY, "skill_job.csv"), index=False)

    # Get each job requirements
each_job_requirements = neo4j_utils.find_each_job_requirement()

rows = []
for path in each_job_requirements:
    row = {
        "job": path["job"]["name"],
        "job_skill": path["job_skill"]["name"]
    }
    rows.append(row)
df = pd.DataFrame(rows)
df.to_csv(os.path.join(COMPILED_GRAPH_DIRECTORY, "job_requirement.csv"), index=False)